## <b> <span style='color:#2ae4f5'>|</span> Table of Contents </b>

* [L-shape Algorithm](#1)

	* [Exercise Page 235](#2)

* [Scenario Generation](#4)

	* [Inverse Transform Sampling](#5)

	* [Scenario Reduction](#6)

	* [Clustering](#7)


# <b>1 <span style='color:#2ae4f5'>|</span> L-shape Algorithm <a id = "1" > </b> 

`Problem Decomposition`

- **Master Problem**: Solve the master problem, which involves the first stage decision variables.

- **Subproblems**: For each scenario, solve the subproblems using the fixed variables from the master problem.

`Iterative Process`

* **Step 1**: Solve the master problem to get candidate solutions for the first-stage variables.

* **Step 2**: Solve the subproblems for each scenario using the candidate solutions from the master problem.

* **Step 3**: Check for feasibility and optimality. If the subproblems provide cuts (constraints) that are violated by the master problem solution, add these cuts to the master problem.

* **Step 4**: Update the master problem with the new cuts and resolve.

* **Step 5**: Repeat steps 1-4 until no more cuts are added, indicating convergence to the optimal solution.

## 
<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2ae4f5'>1.1 |</span></b> Exercise Page 235 <a id = "2" ></b></p>
</div>

$min \ z = 100x_1 + 150x_2 +E_{\xi}[q_1y_1+q_2y_2]$

$s.t.$

$x_1 + x_2 \le 120$

$6y_1 + 10y_2 \le 60x_1$

$8y_1 + 5y_2 \le 80x_1$

$ 0 \le y_1 \le d_1$

$ 0 \le y_2 \le d_2$

$ x_1 \ge 40$

$ x_2 \ge 20$

In [3]:
import numpy as np
from pulp import *
import math

In [4]:
idx = [1, 2]
scenarios = [1, 2]
prob = {1: 0.4, 2: 0.6}
d = {1: {1: 500, 2: 300}, 2: {1: 100, 2: 300}}
q = {1: {1: -24, 2: -28}, 2:{1: -28, 2: -32}}

In [5]:
# Define Master problem
master = LpProblem('MasterProblem', sense=LpMinimize)
x = LpVariable.dicts("X", idx, lowBound=0, cat="continuous")
theta = LpVariable('Theta', cat="continuous")

master += 100*x[1] + 150*x[2] + theta
master += x[1] + x[2] <= 120
master += x[1] >= 40
master += x[2] >= 20

In [6]:
# Define Subproblems
h = {}
for s in scenarios:
	sub = LpProblem('SubProblem', LpMinimize)
	y = LpVariable.dicts('Y', idx, lowBound= 0, cat=LpContinuous)
	sub += q[1][s]*y[1] + q[2][s]*y[2]
	sub += 6*y[1] + 10*y[2] <= 60 * x[1]
	sub += 8*y[1] + 5*y[2] <= 80 * x[2]
	sub += y[1] <= d[1][s]
	sub += y[2] <= d[2][s]
 
	h_tmp = []
	for name, constraint in sub.constraints.items():
		h_tmp.append(-constraint.toDict()['constant'])
	h[s] = h_tmp
 
h

{1: [0, 0, 500, 100], 2: [0, 0, 300, 300]}

In [7]:
T = []
for name, constraint in sub.constraints.items():
    coeff_row = []
    for var in master.variables():
        if var != theta:
            coeff = constraint.get(var, 0)
            coeff_row.append(coeff)
    T.append(coeff_row)
    
T

[[-60, 0], [0, -80], [0, 0], [0, 0]]

In [8]:
v = 0
master = LpProblem('MasterProblem', sense=LpMinimize)
x = LpVariable.dicts("X", idx, lowBound=0, cat="continuous")
theta = LpVariable('Theta', cat="continuous")

master += 100*x[1] + 150*x[2] + theta
master += x[1] + x[2] <= 120
master += x[1] >= 40
master += x[2] >= 20

v += 1
master.solve(PULP_CBC_CMD(msg=False))

print(f"Master Problem Objective Function: {value(master.objective)}")

if v == 1:
	theta.varValue = -math.inf

print(f"Theta: {theta.varValue}")

Master Problem Objective Function: 7000.0
Theta: -inf


In [9]:
e = []
E = []
simplex_multiplier = {}
for s in scenarios:
	sub = LpProblem('SubProblem', LpMinimize)
	y = LpVariable.dicts('Y', idx, lowBound= 0, cat=LpContinuous)
	sub += q[1][s]*y[1] + q[2][s]*y[2]
	sub += 6*y[1] + 10*y[2] <= 0 + 60 * x[1].varValue
	sub += 8*y[1] + 5*y[2] <= 0 + 80 * x[2].varValue
	sub += y[1] <= d[1][s]
	sub += y[2] <= d[2][s]
	
	sub.solve(PULP_CBC_CMD(msg=False, keepFiles=False))
	
	pi = []
	for _, c in sub.constraints.items():
		pi.append(c.pi)
	simplex_multiplier[s] = pi
	
	print(f"Scenario {s}: y1: {y[1].varValue}, y2: {y[2].varValue}")
	print(f"Scenario {s}: pi: {simplex_multiplier[s]}")
	print('---'*10)
 
	e.append(prob[s] * np.array([simplex_multiplier[s]]) @ np.array(h[s]))
	
	E.append(prob[s] * np.array([simplex_multiplier[s]]) @ np.array(T))
    

Scenario 1: y1: 137.5, y2: 100.0
Scenario 1: pi: [0.0, -3.0, 0.0, -13.0]
------------------------------
Scenario 2: y1: 80.0, y2: 192.0
Scenario 2: pi: [-2.32, -1.76, 0.0, 0.0]
------------------------------


In [10]:
print(e)
print(E)
print('---'*10)
e_sum = sum(e)
E_sum = sum(E)
print(f"e: {e_sum}, E: {E_sum}")
w = e_sum - E_sum @ np.array([x[1].varValue, x[2].varValue])
print(f"w: {w}")

[array([-520.]), array([0.])]
[array([[ 0., 96.]]), array([[83.52, 84.48]])]
------------------------------
e: [-520.], E: [[ 83.52 180.48]]
w: [-7470.4]


In [11]:
# Step 1: set r, s, v
r, s, v = (0, 0, 0)

# Step 2: Define Master problem
master = LpProblem('MasterProblem', sense=LpMinimize)
x = LpVariable.dicts("X", idx, lowBound=0, cat="continuous")
theta = LpVariable('Theta', cat="continuous")

master += 100*x[1] + 150*x[2] + theta
master += x[1] + x[2] <= 120
master += x[1] >= 40
master += x[2] >= 20

not_stop_condition = True
while not_stop_condition:
    v += 1
    master.solve(PULP_CBC_CMD(msg=False))
    
    print(f'Iteration {v}')
    print(f"Master Problem Objective Function: {value(master.objective)}")

    if v == 1:
        theta.varValue = -math.inf
    print(f"Theta: {theta.varValue}")
    
    
    
        
    e = []
    E = []
    simplex_multiplier = {}
    for s in scenarios:
        sub = LpProblem('SubProblem', LpMinimize)
        y = LpVariable.dicts('Y', idx, lowBound= 0, cat=LpContinuous)
        sub += q[1][s]*y[1] + q[2][s]*y[2]
        sub += 6*y[1] + 10*y[2] <= 0 + 60 * x[1].varValue
        sub += 8*y[1] + 5*y[2] <= 0 + 80 * x[2].varValue
        sub += y[1] <= d[1][s]
        sub += y[2] <= d[2][s]
        
        sub.solve(PULP_CBC_CMD(msg=False, keepFiles=False))
        
        pi = []
        for _, c in sub.constraints.items():
            pi.append(c.pi)
        simplex_multiplier[s] = pi

        e.append(prob[s] * np.array([simplex_multiplier[s]]) @ np.array(h[s]))
        
        E.append(prob[s] * np.array([simplex_multiplier[s]]) @ np.array(T))
     
    e_sum = sum(e)
    E_sum = sum(E)
    w = e_sum - E_sum @ np.array([x[1].varValue, x[2].varValue])
    if w > theta.varValue:
        s += 1
        master.addConstraint(list(E_sum @ np.array([x[1], x[2]]))[0] + theta >= e_sum[0])
        print("added optima1ity cut to step 1.")
        print('***'*15)
    else:
        print('***'*15)
        print('STOP CONDITION')
        print('***'*15)
        print(f"Master Problem Objective Function: {value(master.objective)}")
        print(f"x1: {x[1].varValue}, x2: {x[2].varValue}")
        not_stop_condition = False

Iteration 1
Master Problem Objective Function: 7000.0
Theta: -inf
added optima1ity cut to step 1.
*********************************************
Iteration 2
Master Problem Objective Function: -2299.2000000000007
Theta: -18299.2
added optima1ity cut to step 1.
*********************************************
Iteration 3
Master Problem Objective Function: -1039.3750999999993
Theta: -15697.994
added optima1ity cut to step 1.
*********************************************
Iteration 4
Master Problem Objective Function: -889.5
Theta: -9952.0
added optima1ity cut to step 1.
*********************************************
Iteration 5
Master Problem Objective Function: -855.8333000000002
Theta: -10960.0
*********************************************
STOP CONDITION
*********************************************
Master Problem Objective Function: -855.8333000000002
x1: 46.666667, x2: 36.25


# <b>2 <span style='color:#2ae4f5'>|</span> Scenario Generation <a id = "4" > </b> 

In [12]:
import numpy as np
from pulp import *
import random
import math

## 
<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2ae4f5'>2.1 |</span></b> Inverse Transform Sampling <a id = "5" ></b></p>
</div>

The problem that the inverse transform sampling method solves is as follows:

- Let $X$ be a random variable whose distribution can be described by the cumulative distribution function $F_x$

- We want to generate values of $X$ which are distributed according to this distribution.

The inverse transform sampling method works as follows:

- **Step 1**: Generate a random number u from the standard uniform distribution in the interval [0, 1] i.e. from $U \in [0, 1]$.

- **Step 2**: Find the generalized inverse of the desired CDF, $F^{-1}_{X}(U)$

ref: https://en.wikipedia.org/wiki/Inverse_transform_sampling


Normal Distribution: $F^{-1}(p) = \mu + \sqrt2 \sigma + erf^{-1}(2p-1)$

In [15]:
from scipy.stats import norm

def inverse_cdf_normal(mean, std_dev, p):
    return norm.ppf(p, loc=mean, scale=std_dev)

In [16]:
n = 100
miu_wheat, sigma_wheat = 2.5, 0.5
miu_corn, sigma_corn = 3, 0.6
miu_sugar, sigma_sugar = 20, 4
num_of_kisi = 3

random_list = np.zeros(n - 1)
scenarios = np.zeros((n, num_of_kisi))
probability = np.zeros(n)

# Generating random numbers for n scenarios
for i in range(n - 1):
	random_list[i] = random.random()

sorted_numbers = np.sort(random_list)

# Generating scenarios from randomNumbers and assign its probability
for i, prob in enumerate(sorted_numbers):
	if i == 0:
		probability[i] = prob
		scenarios[i][0] = inverse_cdf_normal(mean=miu_wheat, std_dev=sigma_wheat, p=prob/2)
		scenarios[i][1] = inverse_cdf_normal(mean=miu_corn, std_dev=sigma_corn, p=prob/2)
		scenarios[i][2] = inverse_cdf_normal(mean=miu_sugar, std_dev=sigma_sugar, p=prob/2)
  
	elif i == n - 2:
		probability[i] = (sorted_numbers[i] - sorted_numbers[i - 1])
		central_prob = (sorted_numbers[i] + sorted_numbers[i - 1]) / 2
		scenarios[i][0] = inverse_cdf_normal(mean=miu_wheat, std_dev=sigma_wheat, p=central_prob)
		scenarios[i][1] = inverse_cdf_normal(mean=miu_corn, std_dev=sigma_corn, p=central_prob)
		scenarios[i][2] = inverse_cdf_normal(mean=miu_sugar, std_dev=sigma_sugar, p=central_prob)
		probability[n - 1] = (1 - sorted_numbers[n - 2])
		central_prob = (1 + sorted_numbers[n - 2]) / 2
		scenarios[n - 1][0] = inverse_cdf_normal(mean=miu_wheat, std_dev=sigma_wheat, p=central_prob)
		scenarios[n - 1][1] = inverse_cdf_normal(mean=miu_corn, std_dev=sigma_corn, p=central_prob)
		scenarios[n - 1][2] = inverse_cdf_normal(mean=miu_sugar, std_dev=sigma_sugar, p=central_prob)
  
	else:
		probability[i] = (sorted_numbers[i] - sorted_numbers[i - 1])
		central_prob = (sorted_numbers[i] + sorted_numbers[i - 1]) / 2
		scenarios[i][0] = inverse_cdf_normal(mean=miu_wheat, std_dev=sigma_wheat, p=central_prob)
		scenarios[i][1] = inverse_cdf_normal(mean=miu_corn, std_dev=sigma_corn, p=central_prob)
		scenarios[i][2] = inverse_cdf_normal(mean=miu_sugar, std_dev=sigma_sugar, p=central_prob)

In [17]:
total_scenario, total_probabilities = scenarios, probability

In [18]:
total_scenario, sum(total_probabilities)

(array([[ 1.42239577,  1.70687493, 11.37916618],
        [ 1.59612121,  1.91534546, 12.7689697 ],
        [ 1.7150535 ,  2.0580642 , 13.72042797],
        [ 1.81022653,  2.17227183, 14.4818122 ],
        [ 1.8397662 ,  2.20771944, 14.71812961],
        [ 1.8515946 ,  2.22191352, 14.81275679],
        [ 1.8632616 ,  2.23591392, 14.90609281],
        [ 1.87389772,  2.24867726, 14.99118174],
        [ 1.87935574,  2.25522688, 15.0348459 ],
        [ 1.88670435,  2.26404523, 15.09363484],
        [ 1.89602211,  2.27522653, 15.16817687],
        [ 1.9180204 ,  2.30162448, 15.34416317],
        [ 1.93956052,  2.32747262, 15.51648415],
        [ 1.94703535,  2.33644241, 15.57628276],
        [ 1.95604639,  2.34725567, 15.64837115],
        [ 1.97628833,  2.371546  , 15.81030666],
        [ 2.00192125,  2.4023055 , 16.01537001],
        [ 2.01749895,  2.42099874, 16.13999159],
        [ 2.02191753,  2.42630104, 16.17534026],
        [ 2.02579344,  2.43095213, 16.20634755],
        [ 2.0304538 

## 
<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2ae4f5'>2.2 |</span></b> Scenario Reduction <a id = "6" ></b></p>
</div>

In [19]:
# Creating the index matrices for each scenario
total_index = np.zeros(n)
for i, scenario in enumerate(total_scenario):
    total_index[i] = i    
total_index

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
       91., 92., 93., 94., 95., 96., 97., 98., 99.])

In [20]:
first_year = {"removed_scenarios": {"scenario": [], "probability": [], "index": []},
              "keeped_scenarios": {"scenario": total_scenario, "probability": total_probabilities, "index": total_index}}

In [21]:
# Define distance matrix to store distance between scenario i & j
minimum_distances = []
distance_matrix = np.zeros((n, n))

# Calculating each indexes of Distance Matrix
for i1, i in enumerate(first_year["keeped_scenarios"]["scenario"]):
	for i2, j in enumerate(first_year["keeped_scenarios"]["scenario"]):
		if i1 != i2:
			# Calcualte distances based on Euclidean Distance
			distance_matrix[i1][i2] = (math.sqrt(math.pow((i[0] - j[0]), 2) + math.pow((i[1] - j[1]), 2) + math.pow((i[2] - j[2]), 2)))
		else:
			distance_matrix[i1][i2] = 10000
   
# print(distance_matrix)

# find smallest (Probability * distance)
for i, distance in enumerate(distance_matrix):
	# Calculating (probability * distance) for each scenario
	minimum_distances.append(first_year["keeped_scenarios"]["probability"][i] * min(distance))

min_value = min(minimum_distances)
min_index = minimum_distances.index(min_value)
first_year["removed_scenarios"]["scenario"].append(first_year["keeped_scenarios"]["scenario"][min_index])
first_year["removed_scenarios"]["probability"].append(first_year["keeped_scenarios"]["probability"][min_index])
first_year["removed_scenarios"]["index"].append(first_year["keeped_scenarios"]["index"][min_index])
first_year["keeped_scenarios"]["scenario"] = np.delete(first_year["keeped_scenarios"]["scenario"], min_index, axis=0)
first_year["keeped_scenarios"]["probability"] = np.delete(first_year["keeped_scenarios"]["probability"], min_index, axis=0)
first_year["keeped_scenarios"]["index"] = np.delete(first_year["keeped_scenarios"]["index"], min_index)

# Define while condition
stop_condition = False
minimum_distances = []

# Stop Condition: # of keeped scenarios <= 15
while not stop_condition:
	total_distances = []
	# Candidate the keeped scenarios
	for i, j in enumerate(first_year["keeped_scenarios"]["scenario"]):
		first_year["removed_scenarios"]["scenario"].append(j)
		first_year["removed_scenarios"]["probability"].append(first_year["keeped_scenarios"]["probability"][i])
		first_year["removed_scenarios"]["index"].append(first_year["keeped_scenarios"]["index"][i])
		np.delete(first_year["keeped_scenarios"]["scenario"], i, 0)
		np.delete(first_year["keeped_scenarios"]["probability"], i, 0)
		keeped_index = np.delete(first_year["keeped_scenarios"]["index"], i, 0)
		# print(keeped_index)
		# Calculating the distance between keeped and removed scenarios
		for i1, k in enumerate(first_year["removed_scenarios"]["index"]):
			distances = []
			for i2, z in enumerate(keeped_index):
				distances.append(distance_matrix[int(k)][int(z)])
			minimum_distances.append(first_year["removed_scenarios"]["probability"][i1] * min(distances))
		# print(minimum_distances)
		total_distances.append(sum(minimum_distances))
		# print(total_distances)
		first_year["removed_scenarios"]["scenario"].pop(-1)
		first_year["removed_scenarios"]["probability"].pop(-1)
		first_year["removed_scenarios"]["index"].pop(-1)
		minimum_distances = []
	# Find the best choice for removing
	min_value = min(total_distances)
	min_index = total_distances.index(min_value)

	# Check the stop condition
	if len(first_year["keeped_scenarios"]["scenario"]) <= 40:
		stop_condition = True
	else:
		first_year["removed_scenarios"]["scenario"].append(first_year["keeped_scenarios"]["scenario"][min_index])
		first_year["removed_scenarios"]["probability"].append(first_year["keeped_scenarios"]["probability"][min_index])
		first_year["removed_scenarios"]["index"].append(first_year["keeped_scenarios"]["index"][min_index])
		first_year["keeped_scenarios"]["scenario"] = np.delete(first_year["keeped_scenarios"]["scenario"], min_index, 0)
		first_year["keeped_scenarios"]["probability"] = np.delete(first_year["keeped_scenarios"]["probability"], min_index, 0)
		first_year["keeped_scenarios"]["index"] = np.delete(first_year["keeped_scenarios"]["index"], min_index, 0)

In [22]:
first_year

{'removed_scenarios': {'scenario': [array([ 2.12041733,  2.54450079, 16.96333861]),
   array([ 2.64998269,  3.17997923, 21.1998615 ]),
   array([ 2.02191753,  2.42630104, 16.17534026]),
   array([ 2.19718868,  2.63662641, 17.57750942]),
   array([ 2.77830288,  3.33396345, 22.22642302]),
   array([ 2.63068259,  3.15681911, 21.04546074]),
   array([ 2.74539522,  3.29447427, 21.9631618 ]),
   array([ 2.50302462,  3.00362954, 20.02419695]),
   array([ 3.42279427,  4.10735312, 27.38235416]),
   array([ 2.0304538 ,  2.43654455, 16.24363037]),
   array([ 2.88970119,  3.46764143, 23.11760956]),
   array([ 1.87935574,  2.25522688, 15.0348459 ]),
   array([ 2.04315914,  2.45179097, 16.34527313]),
   array([ 2.27953149,  2.73543778, 18.23625188]),
   array([ 2.34651135,  2.81581362, 18.77209079]),
   array([ 2.26889295,  2.72267154, 18.15114361]),
   array([ 2.16777984,  2.60133581, 17.3422387 ]),
   array([ 2.12158456,  2.54590147, 16.97267648]),
   array([ 2.50505042,  3.00606051, 20.04040338])

## 
<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2ae4f5'>2.3 |</span></b> Clustering <a id = "7" ></b></p>
</div>

In [23]:
sum(first_year['keeped_scenarios']['probability'])

0.7403283075643748

In [24]:
# create dictionary to store central represenatative and its probability
stage_cluster = {"cluster": first_year["keeped_scenarios"]["scenario"],
					"probability": first_year["keeped_scenarios"]["probability"]}

# Find the best represetative for removed scenario
for i1, j in enumerate(first_year["removed_scenarios"]["scenario"]):
	distances = []
	for i2, k in enumerate(first_year["keeped_scenarios"]["scenario"]):
		distance = math.sqrt(math.pow((j[0] - k[0]), 2) + math.pow((j[1] - k[1]), 2) + math.pow((j[2] - k[2]), 2))
		distances.append(distance)
	min_value = min(distances)
	min_index = distances.index(min_value)
	# modifying the probability of each cluster
	stage_cluster["probability"][min_index] += first_year["removed_scenarios"]["probability"][i1]

In [25]:
sum(first_year['keeped_scenarios']['probability'])

1.0

In [26]:
first_year['keeped_scenarios']

{'scenario': array([[ 1.42239577,  1.70687493, 11.37916618],
        [ 1.59612121,  1.91534546, 12.7689697 ],
        [ 1.7150535 ,  2.0580642 , 13.72042797],
        [ 1.81022653,  2.17227183, 14.4818122 ],
        [ 1.8632616 ,  2.23591392, 14.90609281],
        [ 1.9180204 ,  2.30162448, 15.34416317],
        [ 1.97628833,  2.371546  , 15.81030666],
        [ 2.00192125,  2.4023055 , 16.01537001],
        [ 2.07343263,  2.48811916, 16.58746106],
        [ 2.11066133,  2.5327936 , 16.88529064],
        [ 2.14457103,  2.57348523, 17.15656821],
        [ 2.18757273,  2.62508728, 17.50058185],
        [ 2.2321614 ,  2.67859368, 17.85729121],
        [ 2.25751161,  2.70901394, 18.0600929 ],
        [ 2.29517089,  2.75420507, 18.36136713],
        [ 2.32540716,  2.79048859, 18.60325726],
        [ 2.36029834,  2.83235801, 18.88238676],
        [ 2.41024306,  2.89229168, 19.28194451],
        [ 2.43841625,  2.92609951, 19.50733003],
        [ 2.49038911,  2.98846694, 19.92311292],
        